In [132]:
import numpy as np
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

### Extract All The Links

In [205]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# start at this page
driver.get('https://dialnet.unirioja.es/tesis/fechalectura?registrosPorPagina=20&inicio=450')

In [206]:
def lang_scraper(num=8):
    absLinks=[]
    max_pages = 0
    while True:
        link1 = driver.find_elements_by_css_selector("span.titulo")
        for tags in link1:
            link2 = tags.find_elements_by_tag_name("a")
            for link3 in link2:
                absLinks.append(link3.get_attribute("href"))
        # extract links
        try:
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
            max_pages += 1
            if max_pages == num:
                return absLinks
                break
        # stop if no more pages available
        except NoSuchElementException:
            break  

In [208]:
collect = lang_scraper()

In [192]:
len(collect)

160

### Get Contents From Each Link

In [209]:
%%time
data = []
for link in collect:
    time.sleep(7)
    driver.get(link)

    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text 
    except NoSuchElementException:
        break

    #collect the data
    lang= {'content': language}
    data.append(lang)


Wall time: 21min 24s


In [210]:
en_sp = pd.DataFrame(data)

In [211]:
len(en_sp)

160

### Data Cleaning

In [212]:
# remove \n 
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))

# return only rows with 'Espanol' in the text (because it contains english translate too)
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

# remove rows with 0
en_sp = en_sp[en_sp['Spanish'] != 0]

# create two new variables: Spanish_text, English_text to separate the translations
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])

# remove 'Galego' translation in the English_text variable
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

en_sp['Spanish_text'] = en_sp['Spanish_text'].apply(lambda x: x.split('Español')[-1])

# remove any text that contains 'Español' in the English_text
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: 0 if 'Español' in x else x)

# remove rows with 0
en_sp = en_sp[en_sp['English_text'] != 0]


In [213]:
# make sure Spanish_text and English_text do not have the same value
en_sp = en_sp[en_sp['Spanish_text'] != en_sp['English_text']]


In [214]:
# format dataframe
en_sp.reset_index(inplace=True)
en_sp.drop(['index', 'content', 'Spanish'], axis=1, inplace=True)

In [215]:
len(en_sp)

64

In [216]:
en_sp['Spanish_text'][10]

'La tenacidad de fractura en modo I (KIC) es uno de los parámetros más importantes en la mecánica de fractura aplicada a rocas. KIC es una propiedad intrínseca del material que representa su capacidad para resistir su rotura a tracción en la presencia de un defecto preexistente. La Sociedad Internacional para la Mecánica de Rocas (ISRM) ha sugerido cuatro métodos para determinar KIC, pero todos presentan algunos inconvenientes. El principal objetivo de esta tesis es desarrollar un método de ensayo simple, llamado ‘pseudo-compact tensión’ (pCT), para determinar KIC en rocas mediante muestras cilíndricas ensayadas bajo condiciones de tracción pura. El estudio se basa en los resultados de un número de ensayos llevados a cabo con distintos tipos de roca, tamaños de probeta y longitudes de entalla. Los valores de KIC obtenidos con el ensayo pCT se comparan con los obtenidos con el ensayo ‘semi-circular bend’ (SCB), uno de los métodos sugeridos por la ISRM. En algunos de estos ensayos tambié

In [217]:
en_sp['English_text'][10]

'Mode I fracture toughness (KIC) is one of the most important parameters in rock fracture mechanics. KIC is an intrinsic material property that represents the ability of a material containing a pre-existing defect to resist tensile failure. The International Society for Rock Mechanics (ISRM) has proposed four suggested methods for determining KIC. However, these methods present some drawbacks. The main objective of this thesis is to develop a simple testing approach, called pseudo-compact tension (pCT) test, to measure KIC in rocks using cylindrical single edge-notched specimens loaded in pure tension. The study is based on the results of a large set of fracture toughness experiments performed with different rock types, sample sizes, and a range of notch length ratios. The KIC values derived with the pCT test are compared with those obtained with one of the suggested methods of the ISRM, the semi-circular bend (SCB) test. Some selected pCT and SCB tests were also complemented with the 

In [218]:
en_sp.to_csv('../CSV_files/lang_doc13.csv', index=False)